In [25]:
from bs4 import BeautifulSoup
with open('work_test.htm') as file:
    data = file.read()
divpm = BeautifulSoup(data,'html.parser').find('div', attrs={'id':'Page'}).findAll('div',attrs = {'parameters':True})

total = {}
for i,div in enumerate(divpm):
    total[div['id']] = {'id':div['id'], 'no':i, 'src':div['src']}
    for p in div['parameters'].split(';'):
        total[div['id']].update(dict(zip(p.split(':')[::2], p.split(':')[1::2])))
    for s in div['style'].split('; '):
        total[div['id']].update(dict(zip(s.split(':')[::2], s.split(': ')[1::2])))
    for t in div['hdxproperties'].split(';'):
        total[div['id']].update(dict(zip(t.split(':')[::2], t.split(':')[1::2])))

len(total)

337

In [24]:
# check element numbers 
count= {'all':0}
for i,v in total.items():
    if 'Value?ShapeID' in v :
        #print('{:25} {}'.format(i, v['Value?ShapeID']))
        if v['Value?ShapeID'] in count :
            count[v['Value?ShapeID']] += 1
        else:
            count[v['Value?ShapeID']] = 1
    else:
        #print(i)
        if 'none' in count:
            count['none'] += 1
        else:
            count['none'] = 1
    count['all'] += 1
print(count)

{'all': 337, 'Line': 200, 'none': 33, 'ValveMan': 75, 'CPump': 6, 'TankNozzle': 8, 'BP': 3, 'ValvePseudo': 12}


In [ ]:
from bs4 import BeautifulSoup
with open('work_test.htm') as file:
    data = file.read()
soup = BeautifulSoup(data,'html.parser')

main = soup.find('div', attrs={'id':'Page'})

divpm = main.findAll('div',attrs = {'parameters':True})

total = {}
for i,div in enumerate(divpm):
    total[div['id']] = {'id':div['id'], 'no':i, 'src':div['src']}
    for p in div['parameters'].split(';'):
        total[div['id']].update(dict(zip(p.split(':')[::2], p.split(':')[1::2])))
    for s in div['style'].split('; '):
        total[div['id']].update(dict(zip(s.split(':')[::2], s.split(': ')[1::2])))
    for t in div['hdxproperties'].split(';'):
        total[div['id']].update(dict(zip(t.split(':')[::2], t.split(':')[1::2])))
        
result = {}
for i,each in total.items():
    temp = dict(zip(['type','element','rotation','Value?EndPoint1','Value?EndPoint2'],['','','','','']))
    if 'Value?ShapeID' in each:
        temp['type'] = each['Value?ShapeID']
    if 'Point?element' in each:
        temp['element'] = each['Point?element']
    if 'Rotation' in each:
        temp['rotation'] = each['Rotation']
    if 'Value?EndPoint1' in each:
        temp['endpoint1'] = each['Value?EndPoint1']
    if 'Value?EndPoint2' in each:
        temp['endpoint2'] = each['Value?EndPoint2']
    temp['left'] = each['LEFT'].rstrip('px')
    temp['top'] = each['TOP'].rstrip('px')
    temp['height'] = each['HEIGHT'].rstrip('px')
    temp['width'] = each['WIDTH'].rstrip('px')
    temp.update(dict(zip(['x1','y1','x2','y2'],[0,0,0,0])))
    if temp['type'] == 'Line':
        if each['src'][-11::] == 'MALineU.sha':
            temp['direction'] = 'V'
        else : 
            temp['direction'] = 'H'
        if temp['direction'] == 'H':
            temp['x1'] = int(temp['left'])
            temp['y1'] = int(temp['top'])
            temp['x2'] = int(temp['left']) + int(temp['width'])
            temp['y2'] = temp['y1']
        elif temp['direction'] == 'V':
            temp['x1'] = int(temp['left'])
            temp['y1'] = int(temp['top'])
            temp['x2'] = temp['x1'] 
            temp['y2'] = int(temp['top']) + int(temp['height'])

    elif temp['type'] == 'ValveMan':
        temp['x1'] = int(temp['left'])
        temp['y1'] = int(temp['top']) + int(temp['height'])/2
        temp['x2'] = temp['x1'] + int(temp['width'])
        temp['y2'] = temp['y1']
        if temp['rotation'] == '90' or temp['rotation'] == '270':
            temp['x1'] = temp['x1'] + int(temp['width'])/2
            temp['y1'] = temp['y1'] - int(temp['width'])/2
            temp['x2'] = temp['x2'] - int(temp['width'])/2
            temp['y2'] = temp['y2'] + int(temp['width'])/2   
    result[i] = temp
for i,each in result.items():
    if each['type'] != '':
        each['point1'] = {}
        each['point2'] = {}
        for j,target in result.items():
            if target['type'] != '' and j != i:
                value = ((target['x1'] - each['x1'])**2 + (target['y1'] - each['y1'])**2)**0.5
                if value < 10:
                    each['point1'][j] = {'distance':value, 'point':'1','element':target['element'],'type':target['type']}
                    
                value = ((target['x2'] - each['x1'])**2 + (target['y2'] - each['y1'])**2)**0.5
                if value < 10:
                    each['point1'][j] = {'distance':value, 'point':'2','element':target['element'],'type':target['type']}
                    
                value = ((target['x1'] - each['x2'])**2 + (target['y1'] - each['y2'])**2)**0.5
                if value < 10:
                    each['point2'][j] = {'distance':value, 'point':'1','element':target['element'],'type':target['type']}
                    
                value = ((target['x2'] - each['x2'])**2 + (target['y2'] - each['y2'])**2)**0.5
                if value < 10:
                    each['point2'][j] = {'distance':value, 'point':'2','element':target['element'],'type':target['type']}